In [ ]:
import os
import v20
import pandas as pd
import numpy as np
import json
import base64
import requests
import logging
import time
import mplfinance as mpf
import openai
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, timezone
from dotenv import load_dotenv
from oandapyV20 import API
from oandapyV20.contrib.factories import InstrumentsCandlesFactory
from oandapyV20.endpoints.orders import OrderCreate
from oandapyV20.contrib.requests import MarketOrderRequest, TakeProfitDetails, StopLossDetails
from oandapyV20.exceptions import V20Error

# Load environment variables
load_dotenv()

# OANDA API configuration
access_token = os.getenv('OANDA_API_TOKEN')
account_id = os.getenv('OANDA_ACCOUNT_ID')
api = API(access_token=access_token, environment="practice")

# Set OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')

# Configure logging
logging.basicConfig(level=logging.INFO)
logging.getLogger("oandapyV20").setLevel(logging.WARNING)

# Parameters for real-time data fetching and processing
granularity = 'M5'
instrument = 'EUR_USD'
pair = 'EUR_USD'
timeframe = '5 minutes'
window_size = 576  # Adjust window size for detecting single patterns
step_size = 5      # Adjust step size accordingly

def fetch_forex_data(from_date, to_date, granularity, instrument):
    logging.info(f"Fetching forex data from {from_date} to {to_date} with granularity {granularity} for instrument {instrument}")
    params = {
        "granularity": granularity,
        "from": from_date,
        "to": to_date
    }
    data = []
    try:
        for request in InstrumentsCandlesFactory(instrument=instrument, params=params):
            response = api.request(request)
            if response:
                for candle in response.get('candles'):
                    time = candle.get('time').split('.')[0] + 'Z'
                    rec = {
                        'time': time,
                        'complete': candle['complete'],
                        'open': float(candle['mid']['o']),
                        'high': float(candle['mid']['h']),
                        'low': float(candle['mid']['l']),
                        'close': float(candle['mid']['c']),
                        'volume': candle['volume'],
                    }
                    data.append(rec)
    except Exception as e:
        logging.error(f"An error occurred fetching data: {e}")
    df = pd.DataFrame(data)
    df['time'] = pd.to_datetime(df['time'])
    df.set_index('time', inplace=True)
    return df

def calculate_rsi(data, length=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=length).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=length).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_macd(data, fast_period=12, slow_period=26, signal_period=9):
    fast_ema = data.ewm(span=fast_period, adjust=False).mean()
    slow_ema = data.ewm(span=slow_period, adjust=False).mean()
    macd = fast_ema - slow_ema
    signal = macd.ewm(span=signal_period, adjust=False).mean()
    return macd, signal

def calculate_bollinger_bands(data, window=20, no_of_std=2):
    rolling_mean = data.rolling(window).mean()
    rolling_std = data.rolling(window).std()
    upper_band = rolling_mean + (rolling_std * no_of_std)
    lower_band = rolling_mean - (rolling_std * no_of_std)
    return rolling_mean, upper_band, lower_band

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def analyze_chart_with_gpt4o(chart_image_path, indicators_summary):
    logging.info("Sending data to OpenAI API for analysis")
    base64_image = encode_image(chart_image_path)
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai.api_key}"
    }
    
    prompt_content = {
        "content": "Please analyze the following candlestick chart for EUR/USD and provide a detailed analysis, including detected patterns, predictions, "
                   "and potential trading signals. The chart includes various indicators such as RSI, EMA, MACD, and Bollinger Bands.\n\n"
                   "Provide the analysis in JSON format with the following structure:\n\n"
                   "{\n"
                   "    \"predictions\": [\n"
                   "        {\n"
                   "            \"timeframe\": \"# minutes\",\n"
                   "            \"patterns\": [\n"
                   "                {\"id\": #, \"pattern_detected\": #, \"pattern_name\": \"Pattern Name\", \"confidence_percentage\": ##}\n"
                   "            ],\n"
                   "            \"action\": \"####\",\n"
                   "            \"entry_price\": #.####,\n"
                   "            \"take_profit\": #.####,\n"
                   "            \"stop_loss\": #.####,\n"
                   "            \"deadline_date\": \"####-##-##T##:##:##Z\"\n"
                   "        }\n"
                   "    ]\n}\n\n",
        "image": f"data:image/png;base64,{base64_image}",
        "indicators_summary": indicators_summary
    }
    
    payload = {
        "model": "gpt-4o",
        "messages": [
            {
                "role": "user",
                "content": json.dumps(prompt_content)
            }
        ],
        "max_tokens": 3000
    }
    
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    
    try:
        response_data = response.json()
        return response_data
    except json.JSONDecodeError:
        logging.error("Failed to decode JSON response from OpenAI API")
        return None

def extract_and_place_order(response_data):
    if not response_data or "choices" not in response_data:
        logging.error("Invalid response data")
        return None

    content = response_data["choices"][0]["message"]["content"]
    start_index = content.find('{')
    end_index = content.rfind('}') + 1
    json_content = content[start_index:end_index]
    
    try:
        analysis = json.loads(json_content)
    except json.JSONDecodeError as e:
        logging.error(f"Failed to parse JSON content: {e}")
        return None
    
    predictions = analysis.get("predictions", [])
    
    for prediction in predictions:
        for pattern in prediction.get("patterns", []):
            confidence = pattern.get("confidence_percentage", 0)
            if confidence > 70:
                action = prediction.get("action", "")
                entry_price = prediction.get("entry_price", 0.0)
                take_profit = prediction.get("take_profit", 0.0)
                stop_loss = prediction.get("stop_loss", 0.0)
                deadline_date = prediction.get("deadline_date", "")
                
                logging.info(f"Order Details - Action: {action}, Entry Price: {entry_price}, Take Profit: {take_profit}, Stop Loss: {stop_loss}")
                
                order_details = {
                    'action': action,
                    'entry_price': entry_price,
                    'take_profit': take_profit,
                    'stop_loss': stop_loss,
                    'deadline_date': deadline_date
                }
                response = place_order(order_details)
                
                if 'orderCancelTransaction' in response:
                    logging.info(f"Order {response['orderCancelTransaction']['orderID']} was canceled: {response['orderCancelTransaction']['reason']}")
                return order_details

    logging.info("No patterns with confidence > 70% found")
    return None

def place_order(order_details):
    instrument = "EUR_USD"
    
    mkt_order = MarketOrderRequest(
        instrument=instrument,
        units=-10000 if order_details['action'].upper() == 'SELL' else 10000,
        takeProfitOnFill=TakeProfitDetails(price=order_details['take_profit']).data,
        stopLossOnFill=StopLossDetails(price=order_details['stop_loss']).data
    )
    
    r = orders.OrderCreate(accountID=account_id, data=mkt_order.data)
    try:
        response = api.request(r)
        logging.info(f"Order placed successfully: {response}")
        return response
    except V20Error as e:
        logging.error(f"Error placing order: {e}")
        return {"error": str(e)}

filename = "normal_chart.png"
width, height = 700, 500

while True:
    start_time = (datetime.now(timezone.utc) - timedelta(days=2)).strftime('%Y-%m-%dT%H:%M:%SZ')
    end_time = datetime.now(timezone.utc).strftime('%Y-%m-%dT%H:%M:%SZ')

    prices = fetch_forex_data(start_time, end_time, granularity, instrument)

    prices['RSI'] = calculate_rsi(prices['close'])
    prices['EMA'] = prices['close'].ewm(span=20, adjust=False).mean()
    prices['MACD'], prices['MACD_Signal'] = calculate_macd(prices['close'])
    prices['Bollinger_Mid'], prices['Bollinger_Upper'], prices['Bollinger_Lower'] = calculate_bollinger_bands(prices['close'])

    indicators_summary = {
        "RSI": {
            "latest": prices['RSI'].iloc[-1],
            "max": prices['RSI'].max(),
            "min": prices['RSI'].min(),
            "mean": prices['RSI'].mean()
        },
        "EMA": {
            "latest": prices['EMA'].iloc[-1]
        },
        "MACD": {
            "latest": prices['MACD'].iloc[-1],
            "signal_latest": prices['MACD_Signal'].iloc[-1]
        },
        "Bollinger_Bands": {
            "upper_latest": prices['Bollinger_Upper'].iloc[-1],
            "lower_latest": prices['Bollinger_Lower'].iloc[-1]
        }
    }

    plot_candlestick_chart(prices, filename, width=width, height=height)

    analysis_result = analyze_chart_with_gpt4o(filename, indicators_summary)
    logging.info(f"OpenAI API Analysis Result: {json.dumps(analysis_result, indent=4)}")

    order_details = extract_and_place_order(analysis_result)
    
    if order_details:
        plot_candlestick_chart(prices, filename, width=width, height=height, order_details=order_details)

    logging.info("Waiting for 5 minutes before next run...")
    time.sleep(300)
